In [1]:
import pygame
import random
import math

pygame.init()

# ===================== CONSTANTS =====================
SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 700
ROAD_WIDTH = 150
LANE_WIDTH = ROAD_WIDTH // 3
CAR_WIDTH = 30
CAR_HEIGHT = 50
FPS = 60

# ===================== COLORS =====================
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (110, 110, 110)
DARK_GRAY = (60, 60, 60)
GREEN = (0, 200, 0)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)
BLUE = (0, 120, 255)
ORANGE = (255, 165, 0)
GRASS = (34, 139, 34)

# ===================== TRAFFIC LIGHT =====================
class TrafficLight:
    def __init__(self, x, y, direction):
        self.x = x
        self.y = y
        self.direction = direction
        self.state = "red"

    def draw(self, screen):
        pygame.draw.rect(screen, DARK_GRAY, (self.x - 5, self.y - 100, 10, 100))
        box = pygame.Rect(self.x - 20, self.y - 120, 40, 60)
        pygame.draw.rect(screen, BLACK, box)
        pygame.draw.rect(screen, WHITE, box, 2)

        colors = [
            RED if self.state == "red" else DARK_GRAY,
            YELLOW if self.state == "yellow" else DARK_GRAY,
            GREEN if self.state == "green" else DARK_GRAY
        ]

        for i, c in enumerate(colors):
            pygame.draw.circle(screen, c, (self.x, self.y - 100 + i * 20), 8)

# ===================== TRAFFIC LIGHT CONTROLLER =====================
class TrafficLightController:
    def __init__(self, lights):
        self.lights = lights
        self.order = ["north", "south", "east"]
        self.index = 0
        self.phase = "green"
        self.timer = 0

        self.min_green = 3000
        self.max_green = 9000
        self.yellow_time = 2000
        self.all_red_time = 1000

    def update(self, dt, density):
        self.timer += dt
        current = self.order[self.index]

        for l in self.lights:
            l.state = "red"

        if self.phase == "green":
            for l in self.lights:
                if l.direction == current:
                    l.state = "green"

            gtime = self.min_green + (self.max_green - self.min_green) * density[current]

            if self.timer >= gtime:
                self.phase = "yellow"
                self.timer = 0

        elif self.phase == "yellow":
            for l in self.lights:
                if l.direction == current:
                    l.state = "yellow"

            if self.timer >= self.yellow_time:
                self.phase = "all_red"
                self.timer = 0

        elif self.phase == "all_red":
            if self.timer >= self.all_red_time:
                self.index = (self.index + 1) % len(self.order)
                self.phase = "green"
                self.timer = 0

# ===================== VEHICLE =====================
class Vehicle:
    def __init__(self, lane, vid):
        self.id = vid
        self.lane = lane
        self.speed = random.uniform(2, 4)
        self.max_speed = 5
        self.waiting = 0
        self.fuel = 0

        offset = random.choice([-LANE_WIDTH, 0, LANE_WIDTH])

        if lane == "north":
            self.x = SCREEN_WIDTH//2 + offset
            self.y = -CAR_HEIGHT
            self.angle = 90
            self.color = BLUE
        elif lane == "south":
            self.x = SCREEN_WIDTH//2 + offset
            self.y = SCREEN_HEIGHT + CAR_HEIGHT
            self.angle = -90
            self.color = GREEN
        else:
            self.x = SCREEN_WIDTH + CAR_WIDTH
            self.y = SCREEN_HEIGHT//2 + offset
            self.angle = 180
            self.color = ORANGE

    def dist_to_center(self):
        return math.hypot(self.x - SCREEN_WIDTH//2, self.y - SCREEN_HEIGHT//2)

    def update(self, dt, lights):
        state = "green"
        for l in lights:
            if l.direction == self.lane:
                state = l.state

        if state == "red" and self.dist_to_center() < 130:
            self.speed = max(0, self.speed - 0.4)
            if self.speed < 0.3:
                self.waiting += dt / 1000
        else:
            self.speed = min(self.max_speed, self.speed + 0.1)

        rad = math.radians(self.angle)
        self.x += self.speed * math.cos(rad)
        self.y += self.speed * math.sin(rad)

        self.fuel += self.speed * 0.01

    def off_screen(self):
        return (
            self.x < -200 or self.x > SCREEN_WIDTH + 200 or
            self.y < -200 or self.y > SCREEN_HEIGHT + 200
        )

    def draw(self, screen):
        surf = pygame.Surface((CAR_WIDTH, CAR_HEIGHT), pygame.SRCALPHA)
        pygame.draw.rect(surf, self.color, (0, 0, CAR_WIDTH, CAR_HEIGHT), border_radius=5)
        rot = pygame.transform.rotate(surf, -self.angle)
        screen.blit(rot, rot.get_rect(center=(self.x, self.y)))

# ===================== MAIN SYSTEM =====================
class IntelligentTrafficSystem:
    def __init__(self):
        self.screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
        pygame.display.set_caption("Intelligent Traffic Management System")
        self.clock = pygame.time.Clock()

        self.lights = [
            TrafficLight(SCREEN_WIDTH//2 - 120, SCREEN_HEIGHT//2, "north"),
            TrafficLight(SCREEN_WIDTH//2 + 120, SCREEN_HEIGHT//2, "south"),
            TrafficLight(SCREEN_WIDTH//2, SCREEN_HEIGHT//2 + 120, "east")
        ]

        self.controller = TrafficLightController(self.lights)
        self.vehicles = []
        self.next_id = 1

    # -------- DRAWING (DESIGN FROM SECOND FILE) --------
    def draw_environment(self):
        self.screen.fill(GRASS)

        pygame.draw.rect(self.screen, GRAY,
            (SCREEN_WIDTH//2 - ROAD_WIDTH//2, 0, ROAD_WIDTH, SCREEN_HEIGHT))

        pygame.draw.rect(self.screen, GRAY,
            (SCREEN_WIDTH//2, SCREEN_HEIGHT//2 - ROAD_WIDTH//2,
             SCREEN_WIDTH//2, ROAD_WIDTH))

        pygame.draw.rect(self.screen, DARK_GRAY,
            (SCREEN_WIDTH//2 - ROAD_WIDTH//2,
             SCREEN_HEIGHT//2 - ROAD_WIDTH//2,
             ROAD_WIDTH, ROAD_WIDTH))

        self.draw_lane_markings()
        self.draw_stop_lines()
        self.draw_labels()

    def draw_lane_markings(self):
        for y in range(0, SCREEN_HEIGHT, 35):
            pygame.draw.line(self.screen, WHITE,
                (SCREEN_WIDTH//2 - LANE_WIDTH, y),
                (SCREEN_WIDTH//2 - LANE_WIDTH, y + 20), 3)
            pygame.draw.line(self.screen, WHITE,
                (SCREEN_WIDTH//2 + LANE_WIDTH, y),
                (SCREEN_WIDTH//2 + LANE_WIDTH, y + 20), 3)

        for x in range(SCREEN_WIDTH//2, SCREEN_WIDTH, 35):
            pygame.draw.line(self.screen, WHITE,
                (x, SCREEN_HEIGHT//2 - LANE_WIDTH),
                (x + 20, SCREEN_HEIGHT//2 - LANE_WIDTH), 3)
            pygame.draw.line(self.screen, WHITE,
                (x, SCREEN_HEIGHT//2 + LANE_WIDTH),
                (x + 20, SCREEN_HEIGHT//2 + LANE_WIDTH), 3)

    def draw_stop_lines(self):
        w = ROAD_WIDTH//2
        pygame.draw.line(self.screen, WHITE,
            (SCREEN_WIDTH//2 - w, SCREEN_HEIGHT//2 - w - 5),
            (SCREEN_WIDTH//2 + w, SCREEN_HEIGHT//2 - w - 5), 5)

        pygame.draw.line(self.screen, WHITE,
            (SCREEN_WIDTH//2 - w, SCREEN_HEIGHT//2 + w + 5),
            (SCREEN_WIDTH//2 + w, SCREEN_HEIGHT//2 + w + 5), 5)

        pygame.draw.line(self.screen, WHITE,
            (SCREEN_WIDTH//2 + w + 5, SCREEN_HEIGHT//2 - w),
            (SCREEN_WIDTH//2 + w + 5, SCREEN_HEIGHT//2 + w), 5)

    def draw_labels(self):
        font = pygame.font.Font(None, 36)
        self.screen.blit(font.render("NORTH", True, BLUE), (SCREEN_WIDTH//2 - 40, 20))
        self.screen.blit(font.render("SOUTH", True, GREEN),
                         (SCREEN_WIDTH//2 - 40, SCREEN_HEIGHT - 50))
        self.screen.blit(font.render("EAST", True, ORANGE),
                         (SCREEN_WIDTH - 120, SCREEN_HEIGHT//2 - 20))

    # -------- LOGIC --------
    def density(self):
        d = {"north":0, "south":0, "east":0}
        for v in self.vehicles:
            if v.dist_to_center() < 200:
                d[v.lane] += 1
        return {k:min(v/10,1) for k,v in d.items()}

    def spawn_vehicle(self):
        if random.random() < 0.03:
            lane = random.choice(["north", "south", "east"])
            self.vehicles.append(Vehicle(lane, self.next_id))
            self.next_id += 1

    def run(self):
        last = pygame.time.get_ticks()
        running = True

        while running:
            dt = pygame.time.get_ticks() - last
            last = pygame.time.get_ticks()

            for e in pygame.event.get():
                if e.type == pygame.QUIT:
                    running = False

            self.spawn_vehicle()
            self.controller.update(dt, self.density())

            for v in self.vehicles[:]:
                v.update(dt, self.lights)
                if v.off_screen():
                    self.vehicles.remove(v)

            self.draw_environment()

            for v in self.vehicles:
                v.draw(self.screen)

            for l in self.lights:
                l.draw(self.screen)

            pygame.display.flip()
            self.clock.tick(FPS)

        pygame.quit()

# ===================== RUN =====================
if __name__ == "__main__":
    IntelligentTrafficSystem().run()


C:\Users\Admin\anaconda3\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.13.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
